In [1]:
import env
import pandas as pd
from fredapi import Fred
import requests
import seaborn as sns

In [2]:
# create fred API to acquire information from the Federal Reserve Economic Data
fred = Fred(api_key=env.fredkey)

In [3]:
# The data on unemployment is one of the declines in GDP that could lead to an increasing unemployment rate; 
# therefore, two consecutive declines in GDP would lead to a recession
unem=fred.get_series('UNRATE')
# Industrial Production is the other key point that can be used to forecast the GDP 
# because the decline in the output of the key sector means a decline in the GDP
INDPRO=fred.get_series('INDPRO')
# Real GDP, which reflects inflation
RGDP=fred.get_series('GDPC1')
# GDP
GDP=fred.get_series('GDP')
# whether the National Bureau of Economic Research, 
# which is the independent research organization to announce whether the United Staes is in the Recession officially
is_recession=fred.get_series('USREC')

In [15]:
def gdp_based():
    unem=fred.get_series('UNRATE')
    indpro=fred.get_series('INDPRO')
    rgdp=fred.get_series('GDPC1')
    gdp=fred.get_series('GDP')
    is_recession=fred.get_series('USREC')
    gdp_based_df=pd.DataFrame(data=[unem,indpro,rgdp,gdp,is_recession],index=['unem','indpro','rgdp','gdp','is_recession']).T
    gdp_based_df['quarterly'] = pd.PeriodIndex(gdp_based_df.index, freq='q')
    gdp_based_df=gdp_based_df.groupby('quarterly').agg('mean')
    gdp_based_df.is_recession[round(gdp_based_df.is_recession,2)==0.67]=1
    gdp_based_df.is_recession[round(gdp_based_df.is_recession,2)==0.33]=1
    gdp_based_df['gdp_pct_change']=gdp_based_df.rgdp.pct_change()
    gdp_based_df=gdp_based_df.dropna()
    gdp_based_df.index=gdp_based_df.index.astype('datetime64')
    return gdp_based_df

In [16]:
gdp_based_df=gdp_based

In [9]:
def split_gdp_based(gdp_based_df):
    train_size = int(len(gdp_based_df) * .5)
    validate_size = int(len(gdp_based_df) * .25)
    test_size = int(len(gdp_based_df) - train_size - validate_size)
    validate_end_index = train_size + validate_size
    gdp_train = gdp_based_df[: train_size]
    gdp_validate = gdp_based_df[train_size : validate_end_index]
    gdp_test = gdp_based_df[validate_end_index :]
    gdp_train_val=pd.concat([gdp_train,gdp_validate],axis=0)
    return gdp_train,gdp_validate,gdp_test,gdp_train_val

In [19]:
# General data based

In [22]:
# Use general economy data such as CPI, money supply, Core CPI, GDP deflator, unemployment rate, 
# industrial production, PCE to determine the recession
fred = Fred(api_key=env.fredkey)

In [23]:
def gen_based():
    cpi=fred.get_series('CPIAUCSL')
    moneysup=fred.get_series('M2REAL')
    corecpi=fred.get_series('CPILFESL')
    gdp_def=fred.get_series('A191RI1Q225SBEA')
    unem=fred.get_series('UNRATE')
    indpro=fred.get_series('INDPRO')
    rgdp=fred.get_series('GDPC1')
    gdp=fred.get_series('GDP')
    is_recession=fred.get_series('USREC')
    gdp_based_df=pd.DataFrame(data=[cpi,moneysup,corecpi,gdp_def,unem,indpro,rgdp,gdp,is_recession],index=['cpi','moneysup','corecpi','gdp_def','unem','indpro','rgdp','gdp','is_recession']).T
    gdp_based_df['quarterly'] = pd.PeriodIndex(gdp_based_df.index, freq='q')
    gdp_based_df=gdp_based_df.groupby('quarterly').agg('mean')
    gdp_based_df.is_recession[round(gdp_based_df.is_recession,2)==0.67]=1
    gdp_based_df.is_recession[round(gdp_based_df.is_recession,2)==0.33]=1
    gdp_based_df['gdp_pct_change']=gdp_based_df.rgdp.pct_change()
    gdp_based_df=gdp_based_df.dropna()
    gdp_based_df.index=gdp_based_df.index.astype('datetime64')
    return gen_based_df

In [ ]:
def split_gen_based(gen_based_df):
    train_size = int(len(gen_based_df) * .5)
    validate_size = int(len(gen_based_df) * .25)
    test_size = int(len(gen_based_df) - train_size - validate_size)
    validate_end_index = train_size + validate_size
    gen_train = gen_based_df[: train_size]
    gen_validate = gen_based_df[train_size : validate_end_index]
    gen_test = gen_based_df[validate_end_index :]
    gen_train_val=pd.concat([gen_train,gen_validate],axis=0)
    return gen_train,gen_validate,gen_test,gen_train_val